In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

evaluation_metrics = {
    'RMSE': mean_squared_error,
    'MAE': mean_absolute_error,
    'R-squared': r2_score
}

regularization_techniques = ['none', 'dropout', 'L2']

data_splitting_strategies = {
    'TimeSeriesSplit': [(0.6, 0.2, 0.2)],
    'CustomSplit1': [(0.6, 0.2, 0.2)],
    'CustomSplit2': [(0.7, 0.15, 0.15)]
}

param_grid = {
    'neurons_lstm_1': [32, 64, 128],
    'neurons_lstm_2': [16, 32, 64],
    'dropout_rate': [0.2, 0.4, 0.6],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Define your data preparation here

def create_model(neurons_lstm_1=64, neurons_lstm_2=32, dropout_rate=0.2, learning_rate=0.001):
    model = keras.Sequential([
        layers.LSTM(neurons_lstm_1, activation='relu', input_shape=(num_features, 1), return_sequences=True),
        layers.LSTM(neurons_lstm_2, activation='relu'),
        layers.Dense(1, activation='linear')
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Rest of your code

best_hyperparameters = None
best_metric = float('inf')

# Perform grid search and iterate through different options
for eval_metric_name, eval_metric_func in evaluation_metrics.items():
    for regularization in regularization_techniques:
        for split_name, split_strategy in data_splitting_strategies.items():
            for split_params in split_strategy:
                train_ratio, val_ratio, test_ratio = split_params
                
                tscv = TimeSeriesSplit(n_splits=int(1 / (val_ratio + test_ratio)))
                
                for train_index, test_index in tscv.split(x):
                    num_train = int(train_ratio * len(train_index))
                    num_val = int(val_ratio * len(train_index))
                    num_test = len(test_index)
                    
                    train_index = train_index[-num_train:]
                    val_index = train_index[-num_val:]
                    
                    x_train, x_val, x_test = x[train_index], x[val_index], x[test_index]
                    y_train, y_val, y_test = y[train_index], y[val_index], y[test_index]
                    
                    model = KerasRegressor(build_fn=create_model, verbose=0)
                    
                    param_grid['neurons_lstm_1'] = [32, 64, 128]  # You can adjust these values
                    param_grid['neurons_lstm_2'] = [16, 32, 64]
                    param_grid['dropout_rate'] = [0.2, 0.4, 0.6]
                    param_grid['learning_rate'] = [0.001, 0.01, 0.1]
                    
                    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
                    grid_result = grid_search.fit(x_train, y_train)
                    
                    best_model = grid_result.best_estimator_
                    
                    # Apply regularization if selected
                    if regularization == 'dropout':
                        best_model.model.add(layers.Dropout(0.2))
                    elif regularization == 'L2':
                        best_model.model.add(layers.Dense(1, kernel_regularizer=keras.regularizers.l2(0.01)))
                    
                    best_model.model.compile(optimizer='adam', loss='mean_squared_error')
                    
                    best_model.model.fit(x_train, y_train, epochs=100, batch_size=16, verbose=0, validation_data=(x_val, y_val))
                    
                    y_pred = best_model.predict(x_test)
                    metric = eval_metric_func(y_test, y_pred)
                    
                    if metric < best_metric:
                        best_metric = metric
                        best_hyperparameters = {
                            'Evaluation Metric': eval_metric_name,
                            'Regularization': regularization,
                            'Data Splitting': split_name,
                            'Train Ratio': train_ratio,
                            'Validation Ratio': val_ratio,
                            'Test Ratio': test_ratio,
                            'Hyperparameters': grid_result.best_params_
                        }

# Print the best combination and its performance metric
print("Best Combination:", best_hyperparameters)
print("Best Metric:", best_metric)
